In [1]:
import os
import javabridge
import bioformats
import tifffile
import numpy as np

import tkinter as tk
from tkinter import filedialog
from imaris_ims_file_reader.ims import ims
javabridge.start_vm(class_path=bioformats.JARS)
from aicsimageio import AICSImage

import matplotlib.pyplot as plt

# Load functions


In [2]:
def choose_home_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path

def get_session_name():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askopenfilename(initialdir= "/", title='Please select a movie')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path

def writeTimePoints(sessionName):
    movieFileName = sessionName.split('.dv')[0].split('/')[-1]
    extension = '.tif'
    pathToSaveFrames = os.path.join(sessionName.split('.dv')[0])
    try:
        os.makedirs(pathToSaveFrames)
    except FileExistsError:
       # directory already exists
       pass


    for timePoint in range(0,total_frames):
        newimage = bioformats.load_image(sessionName, c=None, z=0, t=timePoint, series=None, index=None, rescale=False, wants_max_intensity=False, channel_names=None)
        sp = newimage.shape
        newimage=newimage.reshape(1,sp[0],sp[1])


        for z in range(1,zstack_number):    
            image = bioformats.load_image(sessionName, c=None, z=z, t=timePoint, series=None, index=None, rescale=False, wants_max_intensity=False, channel_names=None)

            image=image.reshape(1,sp[0],sp[1])

            newimage = np.concatenate([newimage,image], axis=0).astype('uint16')
        with tifffile.TiffWriter(os.path.join(pathToSaveFrames,movieFileName+str(f"_t{timePoint:03}")+extension), imagej=True) as tif:
            tif.write(newimage)

def writeTimePointsIMS(imagePath):
    outfolder=os.path.splitext(imagePath)[0] #put the foldername of the movie not session
    imsData = ims(imagePath)
    try:
        os.makedirs(outfolder)
    except FileExistsError:
       # directory already exists
       pass

    getFramesFromMoviesIMS(imagePath, startTime=0,stopTime=imsData.shape[0],numberOfZFrames=imsData.shape[2], getMaxProjection=False, outpath=outfolder, extension='.tif')

def getFramesFromMoviesIMS(imagePath, startTime=0, stopTime=25, numberOfZFrames=13, getMaxProjection=False, outpath='./', extension='.tif'):
    moviePathName = os.path.join('/', '/'.join(imagePath.split('/')[:-1]))
    movieFileName = imagePath.split('/')[-1].split('.ims')[0]
    imsData = ims(imagePath)
    for timePoint in range(startTime,stopTime):
        
        tFrame = imsData[timePoint,:,:,:,:]
        tFrameShape = tFrame.shape

        tFrame = tFrame.reshape(1,1,tFrameShape[0],tFrameShape[1],tFrameShape[2]).astype('uint16')
        pathToSaveFrames = os.path.join(outpath, movieFileName)
        with tifffile.TiffWriter(pathToSaveFrames+str(f"_t{timePoint:03}")+extension, imagej=True) as tif:
            tif.write(tFrame)

        if getMaxProjection:
            pathToSaveFrames = os.path.join(outpath, 'MAX_'+movieFileName)
            with tifffile.TiffWriter(pathToSaveFrames+str(f"_t{timePoint:03}")+extension, imagej=True) as tif:
                maxImage = np.max(tFrame, axis=2).astype('uint16')
                tif.write(maxImage)
                
def writeTimePointsTIF(imagePath):
    outfolder=os.path.splitext(imagePath)[0] #put the foldername of the movie not session
    cells = AICSImage(sessionNames[0])
    _,totalTimes, _, zstack, xdim, ydim =  cells.shape
    try:
        os.makedirs(outfolder)
    except FileExistsError:
       # directory already exists
       pass

    getFramesFromMoviesTIFF(imagePath, startTime=0, stopTime=totalTimes, numberOfZFrames=zstack, getMaxProjection=False, outpath=outfolder, extension='.tif')

def getFramesFromMoviesTIFF(imagePath, startTime=0, stopTime=25, numberOfZFrames=13, getMaxProjection=False, outpath='./', extension='.tif'):
    moviePathName = os.path.join('/', '/'.join(imagePath.split('/')[:-1]))
    movieFileName = imagePath.split('/')[-1].split('.tif')[0]
    cells = AICSImage(imagePath)
    cellDask = cells.dask_data
    cellDask1 = cellDask[0]
    imsData = cellDask1[:,0,:,:,:]
    for timePoint in range(startTime,stopTime):
        
        tFrame = imsData[timePoint]
        tFrameShape = tFrame.shape

        tFrame = tFrame.reshape(1,1,tFrameShape[0],tFrameShape[1],tFrameShape[2]).astype('uint16')
        pathToSaveFrames = os.path.join(outpath, movieFileName)
        with tifffile.TiffWriter(pathToSaveFrames+str(f"_t{timePoint:03}")+extension, imagej=True) as tif:
            tif.write(tFrame)

        if getMaxProjection:
            pathToSaveFrames = os.path.join(outpath, 'MAX_'+movieFileName)
            with tifffile.TiffWriter(pathToSaveFrames+str(f"_t{timePoint:03}")+extension, imagej=True) as tif:
                maxImage = np.max(tFrame, axis=2).astype('uint16')
                tif.write(maxImage)

In [3]:
movieFormat = input('Enter the movie type: ')
if movieFormat == '.dv':
    zstack_number= int(input('Enter the number of z slices: '))
    total_frames= int(input('Enter the number of timepoints: '))

Enter the movie type: .ims


# Process bulk


In [4]:
homeFolder = choose_home_folder()
print("Chosen home folder:", homeFolder)

sessionNames = [homeFolder+'/'+i for i in os.listdir(homeFolder) if movieFormat in i and os.path.isfile(os.path.join(homeFolder,i))]

for i in range(len(sessionNames)):
    sessionName = sessionNames[i]
    try:    
        if movieFormat == '.dv':
            writeTimePoints(sessionName)
        if movieFormat == '.ims':
            writeTimePointsIMS(sessionName)
        if movieFormat == '.tif':
            writeTimePointsTIF(sessionName)
    except KeyError or OSError:
        pass

Chosen home folder: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE
Opening readonly file: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06.ims 

Opening readonly file: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06.ims 

Closing file: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06.ims 

Closing file: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06.ims 



# Process 1 movie

In [5]:
sessionName = get_session_name()
if len(list(sessionName))!=0:  
    if movieFormat == '.dv':
        writeTimePoints(sessionName)
    if movieFormat == '.ims':
        writeTimePointsIMS(sessionName)
    if movieFormat == '.tif':
        writeTimePointsTIF(sessionName)

In [ ]:
import session_info
session_info.show()